In [2]:
import rasterio as rio
import numpy as np
from PIL import Image
import math
# core packages
import pandas as pd
from scipy.sparse import csr_matrix

# image packages
import rasterio as rio
import cv2

# visualization packages
import matplotlib.pyplot as plt

In [3]:
def calc_bounds(file, target_coords, length = 10): 
    with rio.open(file) as src:
        data = src.read()
        data = np.squeeze(data).astype("float32")

    lat = target_coords[0]
    long = target_coords[1]
   
    lat_index = np.arange(0, data.shape[0])
    long_index = np.arange(0, data.shape[1])

    A = src.transform

    ymin, xmin = rio.transform.rowcol(A, long, lat - length)
    ymax, xmax = rio.transform.rowcol(A, long + length, lat)

    return data, xmin, xmax, ymin, ymax

def get_coord_string(coords):
    if coords[0] < 0:
        lat = str(coords[0]*-1) + "S" 
    else:
        lat = str(coords[0]) + "N"
    if coords[1] < 0:
        long = str(coords[1]*-1) + "W"
    else:
        long = str(coords[1]) + "E"
    coord_string = lat + long
    
    return coord_string

def segment(file, target_coords, name):
    data, xmin, xmax, ymin, ymax = calc_bounds(file, target_coords)
    print(data, xmin, xmax, ymin, ymax)
   
    seg_data = data[ymax:ymin, xmin:xmax]
    seg_img = Image.fromarray(seg_data)
    
    coords = get_coord_string(target_coords)
    
    path = "../data/raw/segmented/" + str(name) + "_" + str(coords) + ".tif"
    
    try:
        seg_img.save(path)
    except:
        print("Segment was too large to save")
    finally:
        return seg_data

def split_values(segment, default_vals, cumulative_flg):
    res = dict()
    for val in np.unique(segment):
        print("Segmenting: {}".format(val))
        if val in default_vals:
            continue
        if cumulative_flg:
            res[val] = (csr_matrix((segment <= val).astype(int)))
        else:
            res[val] = (csr_matrix((segment == val).astype(int)))
    return res

def grid_data(data, colname, grid_size, smaller):
        
    if smaller:
        grid = cv2.resize(data, dsize=(grid_size, grid_size), interpolation = cv2.INTER_LINEAR)
    else:
        grid = cv2.resize(data, dsize=(grid_size, grid_size), interpolation = cv2.INTER_AREA)

    row = np.arange(grid_size ** 2) // grid_size
    col = np.arange(grid_size ** 2) % grid_size
    
    df = pd.DataFrame(index = row * grid_size + col)
    df["row"] = row
    df["col"] = col
    
    df[colname] = grid.flatten()
    
    return df

the process_data function segments data passed in according to user-specified parameters and returns a dataframe or csv file with the desired data. see parameters below.

- filename: string // path to tif file (oldest data if multiple years of data) 

- target_coords: int tuple (latitude, longtitude) // upper left corner of desired segment 

- grid_size: integer // number of rows/cols in data (sqrt of number of values in dataset)

- name: string // title of dataset, eg. chirps, pop, yield

- years: string (default = "") // must input a list of years of data contained in the set if there's more than one year of data

- agg_data_path: string // path to existing csv file to be merged with new data

- split_vals_flg: boolean // true if we want to split data based on array values (ex. 2 = 2002)

- default_vals: list // contains values that should be ignored when splitting values

- cumulative flag: // used to sum all values less than equal to a value when splitting

In [25]:
def process_data(filename, name, target_coords = (-10, -60), grid_size = 500, years = None, agg_data_path = None, split_vals_flg = False, default_vals = [], cumulative_flg = False):
    if not agg_data_path:
        agg_df = pd.DataFrame()
    else:
        agg_df = pd.read_csv(agg_data_path)
        if name in agg_df.columns:
            print("here")
            agg_df.drop(name, axis = 1, inplace = True)
    
    if years:
        year = years[0]

        for i in range(len(years)):
            file = filename.replace(str(year), str(years[i]))
            col = str(name) + "_" + str(years[i])
            data_seg = segment(file, target_coords, col)
            if (data_seg.shape[0] < grid_size):
                df = grid_data(data_seg, name, grid_size, True)
            else:
                df = grid_data(data_seg, name, grid_size, False)
            if agg_df.empty:
                agg_df = agg_df.append(df)
            else:
                agg_df = pd.merge(agg_df, df)
    else:
        data_seg = segment(filename, target_coords, name)
        print(data_seg.shape)
        # grid for split values data
        if split_vals_flg:
            split_values_dict = split_values(data_seg, default_vals, cumulative_flg)
            for split_val, data in split_values_dict.items():
                if (data.shape[0] < grid_size):
                    df = grid_data(np.array(data.todense()).astype("float32"), "{}_{}".format(name, split_val), grid_size, True)
                else:
                    df = grid_data(np.array(data.todense()).astype("float32"), "{}_{}".format(name, split_val), grid_size, False)
                if agg_df.empty:
                    agg_df = agg_df.append(df)
                else:
                    agg_df = pd.merge(agg_df, df)
        # grid for non split values data
        else:
            if (data_seg.shape[0] < grid_size):
                df = grid_data(data_seg, name, grid_size, True)
            else:
                df = grid_data(data_seg, name, grid_size, False)
            if agg_df.empty:
                agg_df = agg_df.append(df)
            else:
                agg_df = pd.merge(agg_df, df)
    #agg_df.to_csv("../data/processed/datasets/" + name + "_data", index_label = "id")
    if agg_data_path: 
        agg_df.to_csv("../data/processed/datasets/amazon.csv", index = False)
    else:
        agg_df.to_csv("../data/processed/datasets/amazon.csv", index_label = "id")
    return agg_df

In [18]:
GRID_SIZE = 500
# LAT, LONG = ("10S", "60W")
# TARGET_COORDS = (-10, -60)
LAT, LONG = ("0N", "70W")
TARGET_COORDS = (0, -70)
AGG_DATA_PATH = "../data/processed/datasets/amazon.csv"


chirps_path = "../data/raw/chirps/chirps-v2.0.2001.tif"
chirp_years = np.arange(2001, 2019)

pop = "../data/raw/population/gpw_v4_population_count_rev11_2000_30_sec.tif"
pop_years = (2000, 2005, 2010, 2015)

yields = "../data/raw/yields/yield_bean.tif"
yield_years = ("bean","carrot","cassava","chickpea","citrus","coffee","groundnut","maize","soybean","sugarcane","tomato","wheat")

cropland = "../data/raw/sa_cropland.tif"
pastures = "../data/raw/sa_pasture.tif"
#calc slope
elevation = "../data/raw/elevation/srtm_25_15.tif"
elevation_years = ("25_15", "25_16", "26_15", "26_16")

pollution_path = "../data/raw/uvai_05.tif"

tree_cover: values 0-100  
datamask: 1, 2  
defor: 0 - 18  
wdpa: defaults: 1988, 65535

In [46]:
REGION = "amazon"
TARGET_COORDS = (0, -70)
AGG_DATA_PATH = "../data/processed/datasets/amazon.csv"

# yields
bean_path = "../data/raw/yields/yield_bean.tif"
carrot_path = "../data/raw/yields/yield_carrot.tif"
cassava_path = "../data/raw/yields/yield_cassava.tif"
chickpea_path = "../data/raw/yields/yield_chickpea.tif"
citrus_path = "../data/raw/yields/yield_citrus.tif"
coffee_path = "../data/raw/yields/yield_coffee.tif"
groundnut_path = "../data/raw/yields/yield_groundnut.tif"
maize_path = "../data/raw/yields/yield_maize.tif"
soybean_path = "../data/raw/yields/yield_soybean.tif"
sugarcane_path = "../data/raw/yields/yield_sugarcane.tif"
tomato_path = "../data/raw/yields/yield_tomato.tif"
wheat_path = "../data/raw/yields/yield_wheat.tif"

# amazon other
travel_time_path = "../data/raw/{}/travel_time.tif".format(REGION)
dem_path = "../data/raw/{}/dem.tif".format(REGION)
tree_cover_path = "../data/raw/{}/tree_cover.tif".format(REGION)
datamask_path = "../data/raw/{}/datamask.tif".format(REGION)
cropland_path = "../data/raw/{}/cropland.tif".format(REGION)
pasture_path = "../data/raw/{}/pasture.tif".format(REGION)

# amazon year
wdpa_path = "../data/raw/{}/wdpa.tif".format(REGION)
loss_year_path = "../data/raw/{}/loss_year.tif".format(REGION)
# AGG_DATA_PATH
process_data(filename = loss_year_path, 
                  name = "defor", 
                  target_coords = TARGET_COORDS,
                  agg_data_path = AGG_DATA_PATH,
                  split_vals_flg = True,
                  default_vals = [0],
                  cumulative_flg = False)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 4. 4. 6.]
 [0. 0. 0. ... 4. 7. 6.]
 [0. 0. 0. ... 4. 4. 4.]] 0 40000 40000 0
Segment was too large to save
(40000, 40000)
Segmenting: 0.0
Segmenting: 1.0
Segmenting: 2.0
Segmenting: 3.0
Segmenting: 4.0
Segmenting: 5.0
Segmenting: 6.0
Segmenting: 7.0
Segmenting: 8.0
Segmenting: 9.0
Segmenting: 10.0
Segmenting: 11.0
Segmenting: 12.0
Segmenting: 13.0
Segmenting: 14.0
Segmenting: 15.0
Segmenting: 16.0
Segmenting: 17.0
Segmenting: 18.0


,id,row,col,bean,carrot,cassava,chickpea,citrus,coffee,groundnut,...,defor_9.0,defor_10.0,defor_11.0,defor_12.0,defor_13.0,defor_14.0,defor_15.0,defor_16.0,defor_17.0,defor_18.0
0,0,0,0,0.00,0.00,394.00,0.0,0.00,475.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0,1,0.00,0.00,394.00,0.0,0.00,475.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0,2,0.00,0.00,394.10,0.0,0.00,475.10,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,0,3,0.00,0.00,394.34,0.0,0.00,475.34,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,0,4,0.00,0.00,394.58,0.0,0.00,475.58,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,249995,499,495,234.26,342.94,252.68,0.0,397.94,244.68,146.84,...,0.015312,0.036562,0.001094,0.011563,0.010000,0.000000,0.018125,0.054844,0.062656,0.015000
249996,249996,499,496,234.98,344.62,253.64,0.0,399.62,245.64,147.32,...,0.038281,0.027031,0.056875,0.064531,0.060625,0.139219,0.031562,0.102656,0.086562,0.047656
249997,249997,499,497,235.70,346.30,254.60,0.0,401.30,246.60,147.80,...,0.000000,0.021094,0.079531,0.011719,0.002031,0.025625,0.011563,0.160313,0.101562,0.002344
249998,249998,499,498,236.00,347.00,255.00,0.0,402.00,247.00,148.00,...,0.000000,0.011250,0.000000,0.000156,0.001406,0.006875,0.019219,0.016562,0.083906,0.101250


In [1]:
# should be able to do pd.merge like below to put datasets together, but you can also include the path  
# to an aggregate file in the function call and the function will return an aggregate dataset with the 
# new data included 

# pd.merge(chirps_70, pop_70).to_csv("../data/processed/datasets/data_0N70W.csv", index = False)

In [14]:
pd.merge(chirps_70, pop_70).to_csv("../data/processed/datasets/data_0N70W.csv", index = False)